Each dataset was inspected and cleaned independently. this notebook includes some of the functions for data cleaning and feature engineering. The data cleaning process took most of the time spent on this project by far. 

In [ ]:
def translate_price(value):
    value = value.replace('دينار','').replace(',','').replace(' ','')
    value = int(value)
    return value

df['price'] = df['price'].apply(translate_price)

In [ ]:
translation_dict = {
    'إسونس': 'Gasoline',
    'مازوت': 'Diesel',
    'هجين': 'Hybrid',
    'كهربائي': 'Electric'
}

df['نوع الوقود'] = df['نوع الوقود'].replace(translation_dict)

In [ ]:
translation_dict = {
    'يدوي': 'Manual',
    'أوتوماتيكي': 'Automatic'
}

df['ناقل الحركة'] = df['ناقل الحركة'].replace(translation_dict)

In [ ]:
def update_posting_dates(df, time_values):
    df['posting date'] = df['posting date'].astype('object')
    for index, value in enumerate(df['شكل السيارة']):
        if value in time_values:
            df.at[index, 'posting date'] = value

# List of time-related values
time_related_values = [
    'منذ 2 أيام', 'منذ 5 ساعات', 'منذ ساعة', 'منذ 19 ساعات', 'منذ 9 ساعات', 'منذ 10 ساعات', 
    'منذ 4 أيام', 'منذ 24 ساعات', 'منذ يوم', 'منذ شهر', 'منذ 3 أيام', 'منذ 2 أشهر',
    'منذ 6 ساعات', 'منذ 23 أيام', 'منذ 11 أيام', 'منذ 3 أشهر', 'منذ 9 أيام',
    'منذ 7 أيام', 'منذ 5 أيام', 'منذ 13 أيام',
    'منذ 21 أيام', 'منذ 6 أيام', 'منذ 12 أيام', 'منذ 14 أيام', 'منذ 15 أيام',
    'منذ 16 أيام', 'منذ 22 أيام', 'منذ 17 أيام', 'منذ 24 أيام', 'منذ 18 أيام',
    'منذ 19 أيام', 'منذ 25 أيام', 'منذ 20 أيام', 'منذ 26 أيام', 'منذ 6 أشهر',
    'منذ 27 أيام', 'منذ 28 أيام', 'منذ 29 أيام', 'منذ 30 أيام', 'منذ 4 أشهر',
    'منذ 8 أشهر', 'منذ 5 أشهر', 'منذ 8 أيام', 'منذ 10 أيام'
]

# Call the function to update the 'Posting Date' column
update_posting_dates(df, time_related_values)

In [ ]:
k = [ 'Blanc' , 'Noir',  'Gris', 'Rouge' ,'Doré', 'Bleu','Multicolore', 'Argent' , 'منذ شهر','Beige' , 'Vert', 'Berline' ,'Jaune' ,
 'منذ 3 أشهر' , 'منذ 9 أيام' ,'Rose', 'منذ 2 أشهر' ,'Marron']

df = df[~df['قوّة الحصان'].isin(k)]

In [ ]:
v = [ 'منذ 2 أيام' , 'Berline','Compacte', 'منذ 4 أيام' , 'منذ 6 أيام' ,'Pick up',
 'منذ 12 أيام', 'منذ 16 أيام', '4 x 4', 'منذ 20 أيام' ,'منذ 23 أيام', 'منذ شهر',
 'منذ 26 أيام' , 'Utilitaire', 'Cabriolet', 'منذ 2 أشهر', 'منذ 3 أشهر',
 'منذ 8 أيام' ,'منذ 9 أيام', 'منذ 17 أيام','منذ 3 أيام' ,
 'منذ 7 أيام' ,'منذ 11 أيام', 'منذ 13 أيام' ,'منذ 24 أيام', 'منذ 25 أيام',
 'منذ 27 أيام' ,'منذ 4 أشهر']


df = df[~df['اللون'].isin(v)]

In [ ]:
def km(value):
    value = value.replace('كم','').replace(',','').replace(' ','')
    value = int(value)
    return value

df['عدد الكيلومترات']= df['عدد الكيلومترات'].apply(km)

In [ ]:
from datetime import datetime, timedelta

# Set a fixed date variable 
current_date = datetime(2024, 3, 1)

# Function to process the 'posting date' column
def process_posting_date(value):
    # If the value is NaN, return NaT
    if pd.isna(value):
        return pd.NaT

    # Convert the string to lowercase for case-insensitive comparisons
    lower_str = str(value).lower()
    
    # If the string contains 'ساعة' or 'ساعات', set the date to the current date
    if 'ساعة' in lower_str or 'ساعات' in lower_str:
        return current_date
    
    # If the string contains 'شهر' or 'أشهر', calculate the date based on the number of months
    if 'شهر' in lower_str or 'أشهر' in lower_str:
        months_str = ''.join(filter(str.isdigit, lower_str))
        months = int(months_str) if months_str else 0
        return current_date - timedelta(days=months * 30)
    
    # If the string contains 'يوم', calculate the date based on the number of days
    if 'يوم' in lower_str or 'أيام'in lower_str:
        days_str = ''.join(filter(str.isdigit, lower_str))
        days = int(days_str) if days_str else 0
        return current_date - timedelta(days=days)
    
    # If the string contains 'سنة', calculate the date based on the number of years
    if 'سنة' in lower_str:
        years_str = ''.join(filter(str.isdigit, lower_str))
        years = int(years_str) if years_str else 0
        return current_date - timedelta(days=years * 365)
    
    # Return NaT for other cases
    return pd.NaT

# Apply the function to the 'posting date' column
df['posting date2'] = df['posting date'].apply(process_posting_date)

In [ ]:
df['Kilometrage'] = df['Kilometrage'].astype(str)
df['Kilometrage'] = df['Kilometrage'].str.replace(' ', '').str.replace('km', '')
df['Kilometrage'] = pd.to_numeric(df['Kilometrage'], errors='coerce', downcast='integer')

In [ ]:
df['Model'] = df.apply(lambda row: row['Titre'].replace(row['Fabricant'], '').strip(), axis=1)

In [ ]:
def impute_année_nan(df):
    # Replace '97' with '1997' in the 'Année' column
    df['Année'] = df['Année'].replace('97', '1997')
    
    # Convert the 'Année' column to datetime, coercing errors to NaN for non-convertible entries
    df['Année'] = pd.to_datetime(df['Année'], errors='coerce')
    
   
    return df

# Call the function to impute NaN values in the 'Année' column
df = impute_année_nan(df)

# Drop rows with NaN values in the 'Année' column
df = df.dropna(subset=['Année'])

In [ ]:
current_date = datetime(2023, 11, 26)

# Remove 'ago' and leading/trailing whitespaces from the 'Date d'ajout' column
df["Date d'ajout"] = df["Date d'ajout"].apply(lambda x: x.replace('ago', '').strip())

# Function to process the 'Date d'ajout' column
def process_date_dajout(value):
    # Convert the string to lowercase for case-insensitive comparisons
    lower_str = str(value).lower()
    
    # If the string contains 'hour' or 'hours', set the date to the current date
    if 'hour' in lower_str or 'hours' in lower_str: 
        return current_date
    
    # If the string contains 'month' or 'months', calculate the date based on the number of months
    if 'month' in lower_str or 'months' in lower_str:
        months_str = ''.join(filter(str.isdigit, lower_str))
        months = int(months_str) if months_str else 0
        return current_date - timedelta(days=months * 30)
    
    # If the string contains 'day' or 'days', calculate the date based on the number of days
    if 'day' in lower_str or 'days' in lower_str:
        days_str = ''.join(filter(str.isdigit, lower_str))
        days = int(days_str) if days_str else 0
        return current_date - timedelta(days=days)
    
    # Return NaT (Not a Time) for other cases
    return pd.NaT

# Apply the function to the 'Date d'ajout' column
df["Date d'ajout"] = df["Date d'ajout"].apply(process_date_dajout)

In [ ]:
values_to_remove = ['2.0L','4' , '>4.0L','1.2L' ,'3.0L','Essence' ,'<1.0L', '2017' ,'1.3L' ,'2016' ,'1999',
 '2011', '5', '8','2012' ,  '1.6L' ,'2015' ,'1.8L' ,'9' ,'2009', '7', '1.4L', 'Man', '6','7.0', '1.5L','Diesel']

df = df[~df['Marque'].isin(values_to_remove)]

In [ ]:
df = df[(df['Kilométrage'] >= 0)]  # removing negative values 
df = df[~((df['Kilométrage'] <= 50000) & (df['Posting Year'] < 2022))]
df = df[(df['Kilométrage'] < 1) | ((df['Kilométrage'] > 1000) & (df['Kilométrage'] <= 650000))]

In [ ]:
df['prix'] = pd.to_numeric(df['prix'].str.replace('[^0-9]', '', regex=True), errors='coerce')
incorrect_prices = (df['prix'] < 100) & (df['prix'] > 10)
df.loc[incorrect_prices, 'prix'] *= 1000
df = df[df['prix'].apply(lambda x: len(set(str(x))) != 1) & df['prix'].between(10000, 800000)]

In [ ]:
df['Posting Month'] = df['date de publication'].dt.month
df['Posting Year'] = df['date de publication'].dt.year
month_names = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}
df['Posting Month'] = df['Posting Month'].map(month_names)

df = df.drop(columns=['date de publication'])